In [2]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import signal
plt.style.use('seaborn')

In [3]:
root_dir = 'D:/datasets/KAERI_dataset'

train_f = pd.read_csv(os.path.join(root_dir, 'train_features.csv'))
train_t = pd.read_csv(os.path.join(root_dir, 'train_target.csv'))
test_f = pd.read_csv(os.path.join(root_dir, 'test_features.csv'))

In [155]:
def furier(x, fs = 250000, fmax = 375, dt = 1/fs, N = 375):
    df = fmax / N
    f = np.arange(0, N) * df
    xf = np.fft.fft(x) * dt
    #return xf[0:int(N/2 + 1)]
    return np.abs(xf)

In [171]:
ids = train_f['id'].unique()
f_t = []
for _id in ids:
    s1 = furier(train_f.loc[train_f['id'] == _id, 'S1'].values)
    s2 = furier(train_f.loc[train_f['id'] == _id, 'S2'].values)
    s3 = furier(train_f.loc[train_f['id'] == _id, 'S3'].values)
    s4 = furier(train_f.loc[train_f['id'] == _id, 'S4'].values)
    
    f_t.append(np.concatenate([s1, s2, s3, s4]))

f_t = np.array(f_t)

In [170]:
ids = test_f['id'].unique()
test_f_t = []
for _id in ids:
    s1 = furier(test_f.loc[test_f['id'] == _id, 'S1'].values)
    s2 = furier(test_f.loc[test_f['id'] == _id, 'S2'].values)
    s3 = furier(test_f.loc[test_f['id'] == _id, 'S3'].values)
    s4 = furier(test_f.loc[test_f['id'] == _id, 'S4'].values)
    
    test_f_t.append(np.concatenate([s1, s2, s3, s4]))

test_f_t = np.array(test_f_t)

In [193]:
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import KFold

In [230]:
params = {
    'boosting_type':'gbdt',
    'num_leaves':31,
    'max_depth':-1,
    'learning_rate':0.05,
    'n_estimators':2000,
    'objective':'rmse',
    'subsample':0.8,
    'colsample_bytree':0.8,
    'reg_lambda':0.00001,
    'random_state':42,
    'n_jobs':-1
}

In [231]:
nfold = 10
fold = KFold(nfold, shuffle = True, random_state= 25)

In [232]:
predict = {'X':np.zeros(700), 'Y':np.zeros(700), 'M':np.zeros(700), 'V':np.zeros(700)}
loss = {'X':[],'Y':[],'M':[],'V':[]}
for train_idx, val_idx in fold.split(f_t, y = Y):
    for target in ['X','Y','M','V']:
        print(target)
        model = LGBMRegressor(**params)
        
        trainx_fold = f_t[train_idx]
        validx_fold = f_t[val_idx]
        trainy_fold = train_t.iloc[train_idx]
        validy_fold = train_t.iloc[val_idx]
        
        model.fit(trainx_fold, trainy_fold[target], verbose = 500, eval_set=(validx_fold, validy_fold[target]), early_stopping_rounds = 100)

        predict[target] += model.predict(test_f_t)
        #loss[target].append(model.best_score['valid_0']['l2'])

X
Training until validation scores don't improve for 100 rounds
[500]	valid_0's rmse: 68.6798
Early stopping, best iteration is:
[531]	valid_0's rmse: 68.6384
Y
Training until validation scores don't improve for 100 rounds
[500]	valid_0's rmse: 41.3492
[1000]	valid_0's rmse: 41.1059
[1500]	valid_0's rmse: 41.0848
[2000]	valid_0's rmse: 41.079
Did not meet early stopping. Best iteration is:
[1998]	valid_0's rmse: 41.0789
M
Training until validation scores don't improve for 100 rounds
[500]	valid_0's rmse: 5.63926
[1000]	valid_0's rmse: 5.61911
[1500]	valid_0's rmse: 5.61767
Early stopping, best iteration is:
[1525]	valid_0's rmse: 5.61752
V
Training until validation scores don't improve for 100 rounds
[500]	valid_0's rmse: 0.0254622
[1000]	valid_0's rmse: 0.0253884
[1500]	valid_0's rmse: 0.0253786
[2000]	valid_0's rmse: 0.0253759
Did not meet early stopping. Best iteration is:
[2000]	valid_0's rmse: 0.0253759
X
Training until validation scores don't improve for 100 rounds
[500]	valid_0'

In [233]:
submission = pd.read_csv(os.path.join(root_dir, 'sample_submission.csv'))

submission.shape

submission['X'] = predict['X'] / nfold
submission['Y'] = predict['Y'] / nfold
submission['M'] = predict['M'] / nfold
submission['V'] = predict['V'] / nfold

submission.to_csv(os.path.join(root_dir, 'lgb_furier_cv_20200623_2.csv'), index = False)